<a href="https://colab.research.google.com/github/AimjGuytidy/WQU-Thesis/blob/main/GBRT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import sys
sys.version

'3.7.12 (default, Sep 10 2021, 00:21:48) \n[GCC 7.5.0]'

In [4]:
#Import Libraries
import pandas as pd
import numpy as np
import os
#import matplotlib
#import matplotlib.pyplot as plt
import random
# %matplotlib inline
import shutil
from random import shuffle
import itertools
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import Normalizer
from sklearn.multioutput import MultiOutputRegressor
import xgboost as xgb
from datetime import datetime, timedelta
from sklearn import preprocessing

In [5]:
num_periods_output = 24 #to predict
num_periods_input=24 #input

ALL_Test_Data=[]
ALL_Test_Prediction=[]


In [36]:

"""## preprocessing"""

def New_preprocessing(TimeSeries):
   #print(len(TimeSeries))
   Data=[]
   start_date=datetime(2017, 8, 25) # define start date
   for i in range(0,len(ts1)):
      record=[]
      record.append(ts1.iloc[i,:].values[0]) #adding the xchangerate value
      record.append(start_date.month)
      record.append(start_date.day)
      #record.append(start_date.hour)
      #record.append(start_date.minute)
      record.append(start_date.weekday())
      record.append(start_date.timetuple().tm_yday)
      record.append(start_date.isocalendar()[1])
      #print(start_date.month,' ',start_date.day,' ',start_date.hour,' ',start_date.weekday(),' ',start_date.timetuple().tm_yday,' ',start_date.isocalendar()[1])
      start_date=start_date+ timedelta(days=1)
      #print('year',start_date.year,'Month:',start_date.month,' day:',start_date.day,' hour:',start_date.hour)
      Data.append(record)
   ########## change list of lists to df ################
   headers=['pems','month','day','day_of_week','day_of_year','week_of_year']
   Data_df = pd.DataFrame(Data, columns=headers)
   #print(Data_df)
   sub=Data_df.iloc[:,1:]
   New_sub= preprocessing.minmax_scale(sub, feature_range=(-0.5, 0.5))
   #Normalize features to be from -0.5 to 0.5 as mentioned in the paper
   Normalized_Data_df=pd.DataFrame(pd.np.column_stack([Data_df.iloc[:,0],New_sub]), columns=headers)
   #print(Normalized_Data_df)
   #################################################################################################
   # cut training and testing 
   train_split=np.floor(len(Normalized_Data_df)*0.8)   # 60 % training
   #train_split=180
   train_split=int(train_split-(train_split%(num_periods_output+num_periods_input)))
   #print('-------------------',train_split)
   Train=Normalized_Data_df.iloc[0:train_split,:]
   Train=Train.values
   Train = Train.astype('float32')
   print('Traing length :',len(Train))
   total=len(Normalized_Data_df)
   test_split=np.floor(len(Normalized_Data_df)*0.2)   # 20 % testing
   #test_split=20
   #print('-------------------test: ',test_split)
   test_split=int(test_split-(test_split%(num_periods_output+num_periods_input)))
   Test=Normalized_Data_df.iloc[(total-test_split-num_periods_input):,:]
   Test=Test.values
   Test = Test.astype('float32')
   print('Traing length :',len(Test))
   Number_Of_Features=6
   ############################################ Windowing ##################################
   end=len(Train)
   start=0
   next=0
   x_batches=[]
   y_batches=[]  
   count=0
   #print('lennnn',len(Train))
   limit=max(num_periods_input,num_periods_output)
   while next+(limit)<end:
        next=start+num_periods_input
        x_batches.append(Train[start:next,:])
        y_batches.append(Train[next:next+num_periods_output,0])
        start=start+1
   y_batches=np.asarray(y_batches)
   y_batches = y_batches.reshape(-1, num_periods_output, 1) 
   #print('Length of y batches :',len(y_batches),' ',num_periods_input,' ',num_periods_output)
   #print(x_batches)
   x_batches=np.asarray(x_batches) 
   x_batches = x_batches.reshape(-1, num_periods_input, Number_Of_Features)   
   #print('len x_batches ',len(x_batches))
   ############################################ Windowing ##################################
   end_test=len(Test)
   start_test=0
   next_test=0
   x_testbatches=[]
   y_testbatches=[]
   while next_test+(limit)<end_test:
        next_test=start_test+num_periods_input
        x_testbatches.append(Test[start_test:next_test,:])
        y_testbatches.append(Test[next_test:next_test+num_periods_output,0])
        start_test=start_test+num_periods_input
   y_testbatches=np.asarray(y_testbatches)
   y_testbatches = y_testbatches.reshape(-1, num_periods_output, 1)   
   x_testbatches=np.asarray(x_testbatches)
   x_testbatches = x_testbatches.reshape(-1, num_periods_input, Number_Of_Features) 
   #print(' xTestbatches',len(x_testbatches),' yTestbatches',len(y_testbatches))
   return x_batches, y_batches, x_testbatches, y_testbatches

In [9]:
ts = pd.read_csv('/content/oil.csv',header=0,date_parser='Date')
ts.head()

Date  Price
0  Aug 25, 2017  47.65
1  Aug 28, 2017  46.40
2  Aug 29, 2017  46.46
3  Aug 30, 2017  45.96
4  Aug 31, 2017  47.26

In [10]:
ts['Date'] = pd.to_datetime(ts['Date'])
ts.head()

Date  Price
0 2017-08-25  47.65
1 2017-08-28  46.40
2 2017-08-29  46.46
3 2017-08-30  45.96
4 2017-08-31  47.26

In [16]:
ts1 = ts.set_index('Date')
ts1.head()

Price
Date             
2017-08-25  47.65
2017-08-28  46.40
2017-08-29  46.46
2017-08-30  45.96
2017-08-31  47.26

In [18]:
start_date = datetime(2017, 8, 25)
start_date

datetime.datetime(2017, 8, 25, 0, 0)

In [20]:
ts1.loc[start_date,:]

Price    47.65
Name: 2017-08-25 00:00:00, dtype: float64

In [31]:
ts1.iloc[1,:].values[0]

46.4

In [37]:
New_preprocessing(ts1)

Traing length : 768
Traing length : 216


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:29: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead


(array([[[ 4.76500015e+01,  1.36363640e-01,  3.00000012e-01,
           1.66666672e-01,  1.48351654e-01,  1.47058830e-01],
         [ 4.64000015e+01,  1.36363640e-01,  3.33333343e-01,
           3.33333343e-01,  1.51098907e-01,  1.47058830e-01],
         [ 4.64599991e+01,  1.36363640e-01,  3.66666675e-01,
           5.00000000e-01,  1.53846160e-01,  1.47058830e-01],
         ...,
         [ 5.15900002e+01,  2.27272734e-01, -3.33333351e-02,
           1.66666672e-01,  2.06043959e-01,  2.05882356e-01],
         [ 5.21399994e+01,  2.27272734e-01,  0.00000000e+00,
           3.33333343e-01,  2.08791211e-01,  2.05882356e-01],
         [ 5.16199989e+01,  2.27272734e-01,  3.33333351e-02,
           5.00000000e-01,  2.11538464e-01,  2.05882356e-01]],
 
        [[ 4.64000015e+01,  1.36363640e-01,  3.33333343e-01,
           3.33333343e-01,  1.51098907e-01,  1.47058830e-01],
         [ 4.64599991e+01,  1.36363640e-01,  3.66666675e-01,
           5.00000000e-01,  1.53846160e-01,  1.47058830e-01],

In [38]:
data = ts1
x_batches_Full=[]
y_batches_Full=[]
X_Test_Full=[]
Y_Test_Full=[]
for i in range(0,len(data)):
    print('Time series: ',i)
    x_batches=[]
    y_batches=[]
    X_Test=[]
    Y_Test=[]
    TimeSeries=data.iloc[i,:]
    #New_preprocessing(TimeSeries)
    #TimeSeries=np.arange(1,7588,1)
    #print(TimeSeries[7586:])
    x_batches, y_batches,X_Test,Y_Test=New_preprocessing(TimeSeries)          
    for element1 in (x_batches):
        x_batches_Full.append(element1)
            
    for element2 in (y_batches):
        y_batches_Full.append(element2)
                        
    for element5 in (X_Test):
        X_Test_Full.append(element5)
            
    for element6 in (Y_Test):
        Y_Test_Full.append(element6)

Time series:  0
Traing length : 768
Traing length : 216
Time series:  1


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:29: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:29: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead


Traing length : 768
Traing length : 216
Time series:  2
Traing length : 768
Traing length : 216
Time series:  3


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:29: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:29: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead


Traing length : 768
Traing length : 216
Time series:  4
Traing length : 768
Traing length : 216
Time series:  5


...

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:29: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:29: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead


Traing length : 768
Traing length : 216
Time series:  1001
Traing length : 768
Traing length : 216
Time series:  1002


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:29: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:29: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead


Traing length : 768
Traing length : 216
Time series:  1003
Traing length : 768
Traing length : 216
Time series:  1004
Traing length : 768
Traing length : 216


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:29: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead


In [39]:
    #---------------------shuffle windows  X and target Y together-------------------------------------
#print(len(x_batches_Full),'     length of all file : ',len(y_batches_Full))
combined = list(zip(x_batches_Full, y_batches_Full))
random.shuffle(combined)
shuffled_batch_features, shuffled_batch_y = zip(*combined)

#xgboost part
print(len(x_batches_Full))
All_Training_Instances=[]

724605


In [40]:
#=============== flatten each training window into Instance =================================
for i in range(0,len(shuffled_batch_features)):
    hold=[]
    temp=[]
    for j in range(0,len(shuffled_batch_features[i])):
      #**************** to run without features --> comment if else condition **************************
      #print(len(hold))
      if j==(len(shuffled_batch_features[i])-1):
        hold=np.concatenate((hold, shuffled_batch_features[i][j][:]), axis=None)   
      else:
        hold=np.concatenate((hold, shuffled_batch_features[i][j][0]), axis=None)
        
    All_Training_Instances.append(hold)
    

print(len(All_Training_Instances[0]))

29


In [41]:
#=============== flatten each testing window into Instance =================================
All_Testing_Instances=[]
print(len(X_Test_Full))
for i in range(0,len(X_Test_Full)):
  hold=[]
  temp=[]
  for j in range(0,len(X_Test_Full[i])):
       #****************  to run without features --> comment if else condition **************************
       #print(len(hold))
      if j==(len(X_Test_Full[i])-1):
          hold=np.concatenate((hold, X_Test_Full[i][j][:]), axis=None)
      else:
          hold=np.concatenate((hold, X_Test_Full[i][j][0]), axis=None)
   
  All_Testing_Instances.append(hold)

print(len(All_Testing_Instances[0]))

8040
29


In [42]:
#=========================== final shape check =========================
All_Testing_Instances=np.reshape(All_Testing_Instances, (len(All_Testing_Instances),len(All_Testing_Instances[0])))
Y_Test_Full=np.reshape(Y_Test_Full, (len(Y_Test_Full),num_periods_output))

All_Training_Instances=np.reshape(All_Training_Instances, (len(All_Training_Instances),len(All_Training_Instances[0])))
shuffled_batch_y=np.reshape(shuffled_batch_y, (len(shuffled_batch_y),num_periods_output))

In [43]:
#=========================== CALLING XGBOOST ===========================
model=xgb.XGBRegressor(learning_rate =0.07,
 n_estimators=80,
 max_depth=3,
 min_child_weight=1,
 gamma=0.0,
 subsample=0.97,
 colsample_bytree=0.97,
 scale_pos_weight=1,
 seed=42,silent=False)

multioutput=MultiOutputRegressor(model).fit(All_Training_Instances,shuffled_batch_y)


print('Fitting Done!')

/usr/local/lib/python3.7/dist-packages/xgboost/core.py:613: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn("Use subset (sliced data) of np.ndarray is not recommended " +


[08:47:12] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[08:48:50] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[08:50:30] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[08:52:11] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[08:53:56] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[08:55:35] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[08:57:19] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[08:59:03] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of re

In [44]:
multioutput.get_params

<bound method BaseEstimator.get_params of MultiOutputRegressor(estimator=XGBRegressor(base_score=0.5, booster='gbtree',
                                            colsample_bylevel=1,
                                            colsample_bynode=1,
                                            colsample_bytree=0.97, gamma=0.0,
                                            importance_type='gain',
                                            learning_rate=0.07,
                                            max_delta_step=0, max_depth=3,
                                            min_child_weight=1, missing=None,
                                            n_estimators=80, n_jobs=1,
                                            nthread=None,
                                            objective='reg:linear',
                                            random_state=0, reg_alpha=0,
                                            reg_lambda=1, scale_pos_weight=1,
                                        

In [45]:
#============================== PREDICTION ===============================
prediction=multioutput.predict(All_Testing_Instances)

MAPE=np.mean((np.abs(prediction- Y_Test_Full)/np.abs(Y_Test_Full)))
WAPE=np.sum(np.abs(prediction- Y_Test_Full))/np.sum(np.abs(Y_Test_Full))
#print('With Features for {} weeks'.format(No_Of_weeks)) 
#print('MAPE: ',MAPE)
#print('WAPE: ',WAPE)

MSE=np.mean((prediction- Y_Test_Full)**2) 
MAE=np.mean(np.abs((prediction- Y_Test_Full)))
MAPE=np.mean((np.abs(prediction- Y_Test_Full)/np.abs(Y_Test_Full))) 
WAPE=np.sum(np.abs(prediction- Y_Test_Full))/np.sum(np.abs(Y_Test_Full))
print('#~MAE: ',MAE) 
#print('With Features for {} weeks'.format(No_Of_weeks))    
print('MAPE: ',MAPE)   
print('WAPE: ',WAPE)      
 
#print('With Features for {} weeks'.format(No_Of_weeks)) 
print('RMSE: ',MSE**0.5)

#~MAE:  4.269182
MAPE:  0.07202187
WAPE:  0.06905361
RMSE:  5.155122205911877


In [46]:
from sklearn.metrics import r2_score
r2_score(prediction,Y_Test_Full)

0.7435900986567477

In [51]:
pd.DataFrame(All_Testing_Instances)

0          1          2   ...        26        27        28
0     41.369999  39.880001  40.459999  ...  0.166667  0.373626  0.382353
1     42.910000  44.709999  45.580002  ... -0.500000  0.439560  0.460784
2     48.240002  48.349998  47.470001  ...  0.000000 -0.497253 -0.500000
3     56.189999  56.799999  57.950001  ...  0.500000 -0.431319 -0.441176
4     66.019997  65.589996  65.360001  ... -0.166667 -0.365385 -0.362745
...         ...        ...        ...  ...       ...       ...       ...
8035  56.189999  56.799999  57.950001  ...  0.500000 -0.431319 -0.441176
8036  66.019997  65.589996  65.360001  ... -0.166667 -0.365385 -0.362745
8037  63.419998  63.160000  63.330002  ...  0.333333 -0.299451 -0.303922
8038  63.279999  61.950001  63.610001  ... -0.333333 -0.233516 -0.225490
8039  73.110001  73.309998  74.209999  ...  0.166667 -0.167582 -0.166667

[8040 rows x 29 columns]

In [53]:
prediction.shape

(8040, 24)

## Saving the model

In [54]:
import joblib
import pickle

In [55]:
# save the model to disk using joblib
filename = '/content/drive/MyDrive/finalized_model_joblib.sav'
joblib.dump(multioutput, filename)

['/content/drive/MyDrive/finalized_model_joblib.sav']

In [56]:
# save the model to disk using pickle
filename_p = '/content/drive/MyDrive/finalized_model_pickle.sav'
pickle.dump(multioutput, open(filename_p, 'wb'))